# ¿Cómo lidiar con datos faltantes en tus DataFrames?

Es muy común que nuestros DataFrames presenten datos faltantes, antes de empezar a procesar nuestros DataFrames veamos un poco en qué consisten los objetos NaN (Not a Number).

Importemos las librerias Pandas y Numpy para esto:

In [1]:
import numpy as np
import pandas as pd

Un número que no está definido usualmente se representa con el siguiente objeto:

In [2]:
np.nan

nan

¡Este objeto tiene propiedades matemáticas! Al sumar un número, obtenemos como
respuesta el mismo NaN.

In [3]:
np.nan + 0

nan

In [4]:
np.nan > 0

False

La versión 1.0 de pandas incluye un nuevo objeto NA, que es mucho más
general pues, ademas de interactuar con números, tambien puede hacerlo con
cadenas de texto u otras varaibles como las de tipo booleano. Si quieres que
esta nueva definición este incluida entre tus cálculos usa:

In [5]:
pd.options.mode.use_inf_as_na = True

Al sumar NA a una cadena de texto, obtengo el mismo NA:

In [6]:
pd.NA +'Hola mundo'

<NA>

In [7]:
pd.NA | False

<NA>

A continuación, vamos a crear un DataFrame

In [8]:
df = pd.DataFrame(np.arange(0, 15).reshape(5, 3), columns=['a', 'b', 'c'])
df

,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11
4,12,13,14


Y vamos a añadir algunas variables no definidas:

In [9]:
df['d'] = np.nan
df['e'] = np.arange(15, 20)
df.loc[5,:] = pd.NA
df.loc[4,'a'] = pd.NA
df.loc[0,'d'] = 1
df.loc[5,'d'] = 10
df

,a,b,c,d,e
0,0,1,2,1,15
1,3,4,5,NaN,16
2,6,7,8,NaN,17
3,9,10,11,NaN,18
4,<NA>,13,14,NaN,19
5,<NA>,<NA>,<NA>,10,<NA>


Para reconocer cuando un objeto es nulo simplmente usamos:

In [10]:
df.isnull()

,a,b,c,d,e
0,False,False,False,False,False
1,False,False,False,True,False
2,False,False,False,True,False
3,False,False,False,True,False
4,False,False,False,True,False
5,False,False,False,False,False


En dónde todas nuestras variables no definidas fueron marcadas con TRUE,
df.isna() tambien cumple esta función.

Conocer el número de variables nulas por columna puede hacerse juntando el
comando anterior con la funcion de suma:

In [11]:
df.isnull().sum()

a    0
b    0
c    0
d    4
e    0
dtype: int64

Si lo que nos interesa es conocer el número de filas con elementos nulos, basta
con usar axis=1

In [12]:
df.notnull().sum(axis=1)

0    5
1    4
2    4
3    4
4    4
5    5
dtype: int64

O todos los elementos nulos de nuestro DataFrame:

In [13]:
df.size - df.isnull().sum().sum()

26

Reconocer estos elementos nos puede ayudar a filtrar en nuestro DataFrame, en
este caso, me gustaría filtrar por las variables no nulas de la columna ‘a’:

In [14]:
df[df['a'].notnull()]

,a,b,c,d,e
0,0,1,2,1,15
1,3,4,5,NaN,16
2,6,7,8,NaN,17
3,9,10,11,NaN,18
4,<NA>,13,14,NaN,19
5,<NA>,<NA>,<NA>,10,<NA>


`dropna` es perfecto para elimnar rapidamente las filas con registros faltantes:

In [15]:
df.dropna()

,a,b,c,d,e
0,0,1,2,1,15
5,<NA>,<NA>,<NA>,10,<NA>


In [16]:
df[['a']].dropna()

,a
0,0
1,3
2,6
3,9
4,<NA>
5,<NA>


Ya que hemos visto cómo funcionan las variable nulas, veamos cómo lidiar con
ellas. Usando la función fillna podremos reemplazarlas por el valor que
querramos, en este caso 0.

In [17]:
df.fillna(0)

,a,b,c,d,e
0,0,1,2,1,15
1,3,4,5,0,16
2,6,7,8,0,17
3,9,10,11,0,18
4,<NA>,13,14,0,19
5,<NA>,<NA>,<NA>,10,<NA>


Si quisieramos remplazar con el valor siguiente usamos `method="ffill"`

In [18]:
df.fillna(method="ffill")

,a,b,c,d,e
0,0,1,2,1,15
1,3,4,5,1,16
2,6,7,8,1,17
3,9,10,11,1,18
4,<NA>,13,14,1,19
5,<NA>,<NA>,<NA>,10,<NA>


Si quisieramos remplazar con el valor previo usamos `method="bfill"`:

In [19]:
df.fillna(method="bfill")

,a,b,c,d,e
0,0,1,2,1,15
1,3,4,5,10,16
2,6,7,8,10,17
3,9,10,11,10,18
4,<NA>,13,14,10,19
5,<NA>,<NA>,<NA>,10,<NA>


El mismo ejercicio anterior se puede aplicar con las filas usando axis=1:

In [20]:
df.fillna(method="bfill",axis=1)

,a,b,c,d,e
0,0,1,2,1.0,15
1,3,4,5,16.0,16
2,6,7,8,17.0,17
3,9,10,11,18.0,18
4,<NA>,13,14,19.0,19
5,<NA>,<NA>,<NA>,10.0,<NA>


Podemos usar también una serie para reemplazar los valores de una columna en especifico, es importante que haya emparejamiento entre los índices:

In [21]:
fill = pd.Series([100, 101, 102])
fill

0    100
1    101
2    102
dtype: int64

In [22]:
df['d'] = df['d'].fillna(fill)
df['d'] 

0      1.0
1    101.0
2    102.0
3      NaN
4      NaN
5     10.0
Name: d, dtype: float64

In [23]:
df

,a,b,c,d,e
0,0,1,2,1.0,15
1,3,4,5,101.0,16
2,6,7,8,102.0,17
3,9,10,11,NaN,18
4,<NA>,13,14,NaN,19
5,<NA>,<NA>,<NA>,10.0,<NA>


Una de las formas más usadas para reemplazar datos es usar el promedio de las columnas, esto se hace con la función mean. O si se quiere un mejor estimador, usamos median.

In [24]:
df.fillna(df.median())

,a,b,c,d,e
0,0,1,2,1.0,15
1,3,4,5,101.0,16
2,6,7,8,102.0,17
3,9,10,11,55.5,18
4,<NA>,13,14,55.5,19
5,<NA>,<NA>,<NA>,10.0,<NA>


Por último, Pandas también puede interporlar los valores faltanes calculando el valor que puede haber existido en el medio.

In [25]:
df_d = pd.concat([df[['d']], df[['d']].interpolate()],axis=1)
df_d.columns = ['d_antes','d_interpolado']
df_d

,d_antes,d_interpolado
0,1.0,1.000000
1,101.0,101.000000
2,102.0,102.000000
3,NaN,71.333333
4,NaN,40.666667
5,10.0,10.000000


# Cómo lidiar con datos duplicados en Pandas

Es muy usual que los registros de una base de datos aparezcan más de una vez, así que veamos cómo pandas puede ayudarnos a lidiar con estos casos. Para comenzar, importemos pandas y creemos un DataFrame con dos columnas y algunos datos repetidos.

In [26]:
df = pd.DataFrame({'a': ['w'] * 4 + ['x'] * 3 + ['y'] * 2 + ['z']+['v'], 
                   'b': [1, 1, 1, 1, 2, 2, 2, 3, 3, 4,5]})
df

,a,b
0,w,1
1,w,1
2,w,1
3,w,1
4,x,2
5,x,2
6,x,2
7,y,3
8,y,3
9,z,4


Para encontrar los registros duplicados usamos duplicated , que marca con True aquellos casos de filas duplicadas:

In [27]:
df.duplicated()

0     False
1      True
2      True
3      True
4     False
5      True
6      True
7     False
8      True
9     False
10    False
dtype: bool

Podemos usar keep='first' para marcar solo la primera ocurrencia o keep='last' para marcar la última:

In [28]:
df.duplicated(keep='first')

0     False
1      True
2      True
3      True
4     False
5      True
6      True
7     False
8      True
9     False
10    False
dtype: bool

In [29]:
df.duplicated(keep='last')

0      True
1      True
2      True
3     False
4      True
5      True
6     False
7      True
8     False
9     False
10    False
dtype: bool

Identificados los casos duplicados, podemos usar este resultado para filtrar y seleccionar aquellos que no tienen un registro duplicado:

In [30]:
df[~ df.duplicated()]

,a,b
0,w,1
4,x,2
7,y,3
9,z,4
10,v,5


Si quisieras dejar el primer registro de los duplicados o el último, recuerda usar `keep='first' o keep='last'`. Remarco el hecho de que usé negación '~' para ver los registros no duplicados.

Y si me interesara ver cuáles son los registros duplicados, podemos usar `keep=False`

In [31]:
df.duplicated(keep=False)

0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9     False
10    False
dtype: bool

In [32]:
df[df.duplicated(keep=False)]

,a,b
0,w,1
1,w,1
2,w,1
3,w,1
4,x,2
5,x,2
6,x,2
7,y,3
8,y,3


Por último, puedes usar el comando 'drop_duplicates' para eliminar los duplicados. Por defecto, la función guarda el primer resultado keep='first':

In [33]:
df.drop_duplicates()

,a,b
0,w,1
4,x,2
7,y,3
9,z,4
10,v,5


Y si quieres solo borrar duplicados teniendo en cuenta una sola columna, lo puedes hacer mediante una lista nombrando las columnas donde vas a eliminar los duplicados, en este caso, ['a']:

In [34]:
df.drop_duplicates(['a'],keep='last')

,a,b
3,w,1
6,x,2
8,y,3
9,z,4
10,v,5
